# ACCIDENTES DE TRÁFICO - LIMPIEZA DE DATOS

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime 
import calendar

##  HOMOGENEIDAD DE LOS DATASETS

La fuente de la informacion registra los siniestros por año de ocurrencia, utilizando un dataset por año. Uno de los primeros pasos es comparar dichos datasets, comprobando las variables, su formato y categorias para cada variable. Una vez que la información entre los datasets sea homogenea, procederemos a unirlos y crear un unico dataset para el periodo de estudio.

In [2]:
path_acc_2019 = '/home/dsc/Master DS/TFM/GIT/DATOS_BRUTOS/2019_Accidentalidad.csv'
path_acc_2020 = '/home/dsc/Master DS/TFM/GIT/DATOS_BRUTOS/2020_Accidentalidad.csv'
path_acc_2021 = '/home/dsc/Master DS/TFM/GIT/DATOS_BRUTOS/2021_Accidentalidad.csv'

In [3]:
accidents_2019 = pd.read_csv(path_acc_2019, sep = ';' )
accidents_2020 = pd.read_csv(path_acc_2020, sep = ';' )
accidents_2021 = pd.read_csv(path_acc_2021, sep = ';' )

**1. COLUMNAS** - Comprobar si las columnas (variables) coinciden entre los datasets:

In [4]:
columns_acc_2019 = accidents_2019.columns.tolist()
columns_acc_2020 = accidents_2020.columns.tolist()
columns_acc_2021 = accidents_2021.columns.tolist()

In [5]:
columns_acc = columns_acc_2019 + columns_acc_2020 + columns_acc_2021

In [6]:
from collections import Counter

In [7]:
def mycounter(series):
    return Counter(list(series))

In [8]:
columns_freq = mycounter(columns_acc)
columns_freq

Counter({'num_expediente': 3,
         'fecha': 3,
         'hora': 3,
         'localizacion': 3,
         'numero': 3,
         'cod_distrito': 3,
         'distrito': 3,
         'tipo_accidente': 3,
         'estado_meteorológico': 3,
         'tipo_vehículo': 2,
         'tipo_persona': 3,
         'rango_edad': 3,
         'sexo': 3,
         'cod_lesividad': 3,
         'tipo_lesividad': 2,
         'coordenada_x_utm': 3,
         'coordenada_y_utm': 3,
         'positiva_alcohol': 3,
         'positiva_droga': 3,
         'tipo_vehiculo': 1,
         'lesividad': 1})

In [9]:
columns_not_equal = list({x: count for x, count in columns_freq.items() if count < 3})
columns_not_equal

['tipo_vehículo', 'tipo_lesividad', 'tipo_vehiculo', 'lesividad']

In [10]:
columns_equal = list({x: count for x, count in columns_freq.items() if count == 3})
columns_equal

['num_expediente',
 'fecha',
 'hora',
 'localizacion',
 'numero',
 'cod_distrito',
 'distrito',
 'tipo_accidente',
 'estado_meteorológico',
 'tipo_persona',
 'rango_edad',
 'sexo',
 'cod_lesividad',
 'coordenada_x_utm',
 'coordenada_y_utm',
 'positiva_alcohol',
 'positiva_droga']

Cambiamos las siguientes columnas: 
- aquellas donde los datasets utilizan diferentes nombres: 'tipo_lesividad' y 'tipo_vehiculo'
- aquellas con acento, ya que pueden dar problema de procesamiento: 'estado_meteorológico'

In [11]:
accidents_2019 = accidents_2019.rename(columns={'tipo_vehículo':'tipo_vehiculo',
                                               'estado_meteorológico':'estado_meteorologico'})

accidents_2020 = accidents_2020.rename(columns={'tipo_vehículo':'tipo_vehiculo',
                                               'estado_meteorológico':'estado_meteorologico'})

accidents_2021 = accidents_2021.rename(columns={'lesividad':'tipo_lesividad','tipo_vehículo':'tipo_vehiculo',
                                               'estado_meteorológico':'estado_meteorologico'})


**2. NANS Y NULLS / CATEGORIAS** - Comprobar la existencia de Nans y Nulls.

In [12]:
columns_acc_2019 = accidents_2019.columns.str.strip()
columns_acc_2020 = accidents_2020.columns.str.strip()
columns_acc_2021 = accidents_2021.columns.str.strip()

In [13]:
accidents_2019_df = pd . DataFrame (accidents_2019 , columns = columns_acc_2019 )
accidents_2020_df = pd . DataFrame (accidents_2020 , columns = columns_acc_2020 )
accidents_2021_df = pd . DataFrame (accidents_2021 , columns = columns_acc_2021 )

In [14]:
count_nan_2019_acc_df = accidents_2019_df[columns_acc_2019].isna().sum().reset_index()
count_nan_2020_acc_df = accidents_2020_df[columns_acc_2020].isna().sum().reset_index()
count_nan_2021_acc_df = accidents_2021_df[columns_acc_2021].isna().sum().reset_index()

In [15]:
count_null_2019_acc_df = accidents_2019_df[columns_acc_2019].isnull().sum().reset_index()
count_null_2020_acc_df = accidents_2020_df[columns_acc_2020].isnull().sum().reset_index()
count_null_2021_acc_df = accidents_2021_df[columns_acc_2021].isnull().sum().reset_index()

In [16]:
count_nan_null_acc_df = pd.concat([count_nan_2019_acc_df,count_null_2019_acc_df,
                              count_nan_2020_acc_df,count_null_2020_acc_df,
                              count_nan_2021_acc_df, count_null_2021_acc_df],axis=1,
                             keys=['Nan_2019','Null_2019','Nan_2020', 'Null_2020','Nan_2021', 'Null_2021']) 
count_nan_null_acc_df

Nan_2019                    Null_2019         \
                   index      0                 index      0   
0         num_expediente      0        num_expediente      0   
1                  fecha      0                 fecha      0   
2                   hora      0                  hora      0   
3           localizacion      0          localizacion      0   
4                 numero      0                numero      0   
5           cod_distrito      0          cod_distrito      0   
6               distrito      0              distrito      0   
7         tipo_accidente      0        tipo_accidente      0   
8   estado_meteorologico   5130  estado_meteorologico   5130   
9          tipo_vehiculo    176         tipo_vehiculo    176   
10          tipo_persona      0          tipo_persona      0   
11            rango_edad      0            rango_edad      0   
12                  sexo      0                  sexo      0   
13         cod_lesividad  21769         cod_lesividad  21769   
14        tipo_lesividad  21769        tipo_lesividad  21769   
15      coordenada_x_utm      0      coordenada_x_utm      0   
16      coordenada_y_utm      0      coordenada_y_utm      0   
17      positiva_alcohol    109      positiva_alcohol    109   
18        positiva_droga  51649        positiva_droga  51649   

                Nan_2020                    Null_2020         \
                   index      0                 index      0   
0         num_expediente      0        num_expediente      0   
1                  fecha      0                 fecha      0   
2                   hora      0                  hora      0   
3           localizacion      0          localizacion      0   
4                 numero      2                numero      2   
5           cod_distrito      2          cod_distrito      2   
6               distrito      2              distrito      2   
7         tipo_accidente      0        tipo_accidente      0   
8   estado_meteorologico   3196  estado_meteorologico   3196   
9          tipo_vehiculo    137         tipo_vehiculo    137   
10          tipo_persona      0          tipo_persona      0   
11            rango_edad      0            rango_edad      0   
12                  sexo      0                  sexo      0   
13         cod_lesividad  14802         cod_lesividad  14802   
14        tipo_lesividad  14802        tipo_lesividad  14802   
15      coordenada_x_utm      2      coordenada_x_utm      2   
16      coordenada_y_utm      2      coordenada_y_utm      2   
17      positiva_alcohol    102      positiva_alcohol    102   
18        positiva_droga  32351        positiva_droga  32351   

                Nan_2021                    Null_2021         
                   index      0                 index      0  
0         num_expediente      0        num_expediente      0  
1                  fecha      0                 fecha      0  
2                   hora      0                  hora      0  
3           localizacion      0          localizacion      0  
4                 numero      3                numero      3  
5           cod_distrito      3          cod_distrito      3  
6               distrito      3              distrito      3  
7         tipo_accidente      4        tipo_accidente      4  
8   estado_meteorologico   4428  estado_meteorologico   4428  
9          tipo_vehiculo     54         tipo_vehiculo     54  
10          tipo_persona      3          tipo_persona      3  
11            rango_edad      0            rango_edad      0  
12                  sexo      0                  sexo      0  
13         cod_lesividad  18964         cod_lesividad  18964  
14        tipo_lesividad  18964        tipo_lesividad  18964  
15      coordenada_x_utm      9      coordenada_x_utm      9  
16      coordenada_y_utm      9      coordenada_y_utm      9  
17      positiva_alcohol    167      positiva_alcohol    167  
18        positiva_droga  41694        positiva_droga  41694

Del documento de descripción del dataset, sabemos que los Nans registrados en las siguientes variables corresponden a una categoria:
- 'cod_lesividad' --- Sin asistencia sanitaria
- 'tipo_lesividad' --- Sin asistencia sanitaria
- 'positiva_droga' --- Negative

Eliminaremos las siguientes variables, ya que nos ofrecen información no extrapolable:
- 'localizacion'
- 'numero'

Aunque 'estado_meteorologico' tiene un alto numero de Nans, debido a que tiene una informacion relevante con un un alto impacto en nuestro análisis, la vamos a mantener.

Debido a la información redundante de las variables 'cod_distrito' a 'distrito', y 'cod_lesividad' a 'tipo_lesividad', las vamos a eliminar, pero antes vamos a crear dos datasets independientes de normalizacion, donde tendremos la relación cod_distrito-distrito y cod_lesividad-tipo_lesividad.

In [17]:
dist_cod = accidents_2019_df[['distrito','cod_distrito']].drop_duplicates()

In [18]:
lesiv_cod_type = accidents_2019_df[['cod_lesividad','tipo_lesividad']].drop_duplicates()

In [19]:
variables_to_drop = ['distrito','localizacion', 'numero', 'tipo_lesividad']

In [20]:
accidents_2019_df_clean = accidents_2019_df.drop(variables_to_drop, axis=1)
accidents_2020_df_clean = accidents_2020_df.drop(variables_to_drop, axis=1)
accidents_2021_df_clean = accidents_2021_df.drop(variables_to_drop, axis=1)

Una vez que hemos eliminado las variables y antes eliminar los Nans restantes, vamos a analizar cada variable y sus categorias:

In [21]:
list_columns = ['cod_distrito','tipo_accidente','estado_meteorologico','tipo_persona','rango_edad',
                'sexo','positiva_alcohol','tipo_vehiculo', 'cod_lesividad', 'positiva_droga']

In [22]:
def check_feature(feature):
    
    '''Brings a dataframe that checks the features content: lists the categories of the feature, checks if the 
    datasets have the same categories, count each category per dataset and obtain the number of Nans''' 
        
    print('')
    print(feature)
    
    #First check if feature contains same categories
    feature_type_2019 = accidents_2019_df[feature].unique().tolist()
    feature_type_2020 = accidents_2020_df[feature].unique().tolist()
    feature_type_2021 = accidents_2021_df[feature].unique().tolist()
    
    if(set(feature_type_2019) == set(feature_type_2020)):
        if (set(feature_type_2019) == set(feature_type_2021)):
            message ='Same categories'
            print(message)
        else:
            message ='Different categories, lets see if it is because Nans'
            print(message)
    else:
        message ='Different categories, lets see if it is because Nans'
        print(message)
    
    if message == 'Different categories, lets see if it is because Nans':
       
        # When different categories: dataframe of all categories per year
        fea_type_2019_df = pd.DataFrame(feature_type_2019, columns=['Type_2019'])
        fea_type_2020_df = pd.DataFrame(feature_type_2020, columns=['Type_2020'])
        fea_type_2021_df = pd.DataFrame(feature_type_2021, columns=['Type_2021'])
        
        type_fea_19_20 = pd.merge(left=fea_type_2019_df,right=fea_type_2020_df, left_on='Type_2019', 
                                  right_on='Type_2020', how='outer')
        type_fea_19_20_21 = pd.merge(left=type_fea_19_20,right=fea_type_2021_df, left_on='Type_2020',
                                     right_on='Type_2021', how='outer')
                                
        #Dropna and check if the unique lists are still different
        accidents_clean_2019_df = accidents_2019_df.dropna(subset=[feature])
        accidents_clean_2020_df = accidents_2020_df.dropna(subset=[feature])
        accidents_clean_2021_df = accidents_2021_df.dropna(subset=[feature])
        
        fea_type_clean_2019 = accidents_clean_2019_df[feature].unique().tolist()
        fea_type_clean_2020 = accidents_clean_2020_df[feature].unique().tolist()
        fea_type_clean_2021 = accidents_clean_2021_df[feature].unique().tolist()
        
        #print(fea_type_clean_2021)
    
        if(set(fea_type_clean_2019) == set(fea_type_clean_2020)):
            if (set(fea_type_clean_2019) == set(fea_type_clean_2021)):
                message ='Different categories because of Nans'
                print(message)
            else:
                message ='Different categories'
                print(message)
        else:
            message = 'Different categories'
            print(message)
    else:
        message ='Same categories'
        
    
    if message == 'Same categories':
        # Dataframe visualization of all categories
        content_2019_df = accidents_2019_df.groupby(feature)['num_expediente'].count().reset_index()
        content_2020_df = accidents_2020_df.groupby(feature)['num_expediente'].count().reset_index()
        content_2021_df = accidents_2021_df.groupby(feature)['num_expediente'].count().reset_index()
                
        content_df= pd.concat([content_2019_df,content_2020_df,content_2021_df],axis=1,keys=['2019','2020','2021'])
        
        print('The feature has the following categories: ')
        print(content_df)
        
        #Check size

        n_nan_2019= accidents_2019_df[feature].isna().sum()
        n_nan_2020= accidents_2020_df[feature].isna().sum()      
        n_nan_2021 = accidents_2021_df[feature].isna().sum()
        
        size_2019 = accidents_2019_df.shape[0]
        size_2020 = accidents_2020_df.shape[0]
        size_2021 = accidents_2021_df.shape[0]
        
        sum_var_2019 = content_2019_df['num_expediente'].sum()      
        sum_var_2020 = content_2020_df['num_expediente'].sum()       
        sum_var_2021 = content_2021_df['num_expediente'].sum()
              
        check_2019 = size_2019 - sum_var_2019 - n_nan_2019
        check_2020 = size_2020 - sum_var_2020 - n_nan_2020
        check_2021 = size_2021 - sum_var_2021 - n_nan_2021
        
        if check_2019 == 0 & check_2020 == 0 & check_2021 == 0:
            print('Number of Nans in 2019:')
            print (n_nan_2019)
            print('Number of Nans in 2020:')
            print (n_nan_2020)
            print('Number of Nans in 2021:')
            print (n_nan_2021)           
            print('The sum of type of features and Nans is OK to size')
        else:
            print('The sum is NOT OK, analyse more')
       
    if message == 'Different categories because of Nans': 
        # Dataframe visualization of all categories
        content_2019_df = accidents_clean_2019_df.groupby(feature)['num_expediente'].count().reset_index()
        content_2020_df = accidents_clean_2020_df.groupby(feature)['num_expediente'].count().reset_index()
        content_2021_df = accidents_clean_2021_df.groupby(feature)['num_expediente'].count().reset_index()
        
        content_df= pd.concat([content_2019_df,content_2020_df,content_2021_df],axis=1,keys=['2019','2020','2021'])
                
        print('The feature has the following categories: ')
        print(content_df)
        
        #check size
        n_nan_2019= accidents_2019_df[feature].isna().sum()
        n_nan_2020= accidents_2020_df[feature].isna().sum()      
        n_nan_2021 = accidents_2021_df[feature].isna().sum()
        
        size_2019 = accidents_2019_df.shape[0]
        size_2020 = accidents_2020_df.shape[0]
        size_2021 = accidents_2021_df.shape[0]
        
        sum_var_2019 = content_2019_df['num_expediente'].sum()      
        sum_var_2020 = content_2020_df['num_expediente'].sum()       
        sum_var_2021 = content_2021_df['num_expediente'].sum()
              
        check_2019 = size_2019 - sum_var_2019 - n_nan_2019
        check_2020 = size_2020 - sum_var_2020 - n_nan_2020
        check_2021 = size_2021 - sum_var_2021 - n_nan_2021
        
        if check_2019 == 0 & check_2020 == 0 & check_2021 == 0:
            print('Number of Nans in 2019:')
            print (n_nan_2019)
            print('Number of Nans in 2020:')
            print (n_nan_2020)
            print('Number of Nans in 2021:')
            print (n_nan_2021)           
            print('The sum of type of features and Nans is OK to size')
        else:
            print('The sum is NOT OK, analyse more')          
        

In [23]:
for i in list_columns:
    check_feature(i)


cod_distrito
Different categories, lets see if it is because Nans
Different categories because of Nans
The feature has the following categories: 
           2019                        2020                        2021  \
   cod_distrito num_expediente cod_distrito num_expediente cod_distrito   
0             1           2737          1.0           1518          1.0   
1             2           2300          2.0           1351          2.0   
2             3           2687          3.0           1530          3.0   
3             4           4073          4.0           2486          4.0   
4             5           3889          5.0           2084          5.0   
5             6           2383          6.0           1598          6.0   
6             7           2674          7.0           1491          7.0   
7             8           2473          8.0           1786          8.0   
8             9           2683          9.0           1756          9.0   
9            10           25

Del proceso de limpieza y analisis de las variables tenemos las siguientes ideas:

- Hay variables con una **categoria indefinida**:
    - 'tipo_accidente' ---> 'Otros' 
    - 'estado_meteorologico' ---> 'Se desconoce'  
    - 'rango_edad' ---> 'Desconocido' 
    - 'sexo' ---> 'Desconocido'     
    
    
- **Nans** existencia y tratamiento:
    - 'tipo_accidente' ---> los incluimos a la categoria 'Otros'
    - 'estado_meteorologico'---> los incluimos a la categoria 'Se desconoce' 
    - positiva_droga ---> Los Nans son una categoria , los cambiamos por 0
    - debido a la baja cantidad de Nans y que no hay una categoria indefinida, vamos a eliminar las lineas con Nans de las siguientes variables:
        - 'cod_distrito' 
        - 'tipo_persona' 
        - 'positiva_alcohol'
        - 'coordenada_x_utm'
        - 'coordenada_y_utm'
    
    
- **'tipo_vehiculo' y 'cod_lesividad'** tienen diferentes categorias en funcion del dataset , tendremos que hacer los datasets homogeneos. 


- Adicionalmente, comprobaremos que el año en el numero identificativo y el de la fecha:
    
    Según la descripción del dataset, el numero identificativo tiene la siguiente estructura:
    - AAAASNNNNNN
    - AAAA: año
    - S: expediente con siniestro 
    - NNNNNN: numero correlativo al año
    - Hay numeros duplicados , al haber una linea por persona involucrada en el siniestro

In [24]:
accidents_2019_df_clean['ano_fecha'] = pd.DatetimeIndex(accidents_2019_df_clean['fecha']).year.astype(int)
accidents_2019_df_clean['year_exp'] = accidents_2019_df_clean['num_expediente'].str[:4].astype(int)
accidents_2019_df_clean['year_equal_cond'] = accidents_2019_df_clean['year_exp']-(accidents_2019_df_clean['ano_fecha'])
accidents_2019_df_clean.groupby('year_equal_cond')['num_expediente'].count().reset_index()

,year_equal_cond,num_expediente
0,-1,2
1,0,51809


In [25]:
accidents_2020_df_clean['ano_fecha'] = pd.DatetimeIndex(accidents_2020_df_clean['fecha']).year.astype(int)
accidents_2020_df_clean['year_exp'] = accidents_2020_df_clean['num_expediente'].str[:4].astype(int)
accidents_2020_df_clean['year_equal_cond'] =accidents_2020_df_clean['year_exp']-(accidents_2020_df_clean['ano_fecha'])
accidents_2020_df_clean.groupby('year_equal_cond')['num_expediente'].count().reset_index()

,year_equal_cond,num_expediente
0,-1,2
1,0,32431


In [26]:
accidents_2021_df_clean['ano_fecha'] = pd.DatetimeIndex(accidents_2021_df_clean['fecha']).year.astype(int)
accidents_2021_df_clean['year_exp'] = accidents_2021_df_clean['num_expediente'].str[:4].astype(int)
accidents_2021_df_clean['year_equal_cond'] =accidents_2021_df_clean['year_exp']-(accidents_2021_df_clean['ano_fecha'])
accidents_2021_df_clean.groupby('year_equal_cond')['num_expediente'].count().reset_index()

,year_equal_cond,num_expediente
0,-1,5
1,0,41778


In [27]:
indexNames19 = accidents_2019_df_clean[ accidents_2019_df_clean['year_equal_cond'] == -1 ].index
indexNames19

Int64Index([0, 1], dtype='int64')

In [28]:
accidents_2019_df_clean.drop(indexNames19,inplace=True)

In [29]:
accidents_2019_df_clean.drop(['year_equal_cond','year_exp'],axis=1,inplace=True)

In [30]:
indexNames20 = accidents_2020_df_clean[accidents_2020_df_clean['year_equal_cond']==-1].index
accidents_2020_df_clean.drop(indexNames20,inplace=True)
accidents_2020_df_clean.drop(['year_equal_cond','year_exp'],axis=1,inplace=True)

In [31]:
indexNames21 = accidents_2021_df_clean[accidents_2021_df_clean['year_equal_cond']==-1].index
accidents_2021_df_clean.drop(indexNames21,inplace=True)
accidents_2021_df_clean.drop(['year_equal_cond','year_exp'],axis=1,inplace=True)

In [32]:
acc_2019 = accidents_2019_df_clean.dropna(subset=['cod_distrito', 'tipo_persona', 'positiva_alcohol',
                                                  'coordenada_x_utm', 'coordenada_y_utm'])
acc_2020 = accidents_2020_df_clean.dropna(subset=['cod_distrito', 'tipo_persona', 'positiva_alcohol',
                                                  'coordenada_x_utm', 'coordenada_y_utm'])
acc_2021 = accidents_2021_df_clean.dropna(subset=['cod_distrito', 'tipo_persona', 'positiva_alcohol',
                                                  'coordenada_x_utm', 'coordenada_y_utm'])

In [33]:
acc_2019 = acc_2019.fillna({'tipo_accidente': 'Otro', 'estado_meteorologico': 'Se desconoce'})
acc_2020 = acc_2020.fillna({'tipo_accidente': 'Otro', 'estado_meteorologico': 'Se desconoce'})
acc_2021 = acc_2021.fillna({'tipo_accidente': 'Otro', 'estado_meteorologico': 'Se desconoce'})

In [34]:
acc_2019 = acc_2019.fillna({'positiva_droga': 0})
acc_2020 = acc_2020.fillna({'positiva_droga': 0})
acc_2021 = acc_2021.fillna({'positiva_droga': 0})

Vamos a comprobar si aún hay algún Nan...

In [35]:
columns_acc = acc_2019.columns.str.strip()

In [36]:
count_nan_2019_check = acc_2019[columns_acc].isna().sum().reset_index()
count_nan_2020_check = acc_2020[columns_acc].isna().sum().reset_index()
count_nan_2021_check = acc_2021[columns_acc].isna().sum().reset_index()

In [37]:
count_nan_acc_check = pd.concat([count_nan_2019_check,count_nan_2020_check,count_nan_2021_check],axis=1,
                             keys=['Nan_2019','Nan_2020','Nan_2021']) 
count_nan_acc_check

Nan_2019                     Nan_2020         \
                   index      0                 index      0   
0         num_expediente      0        num_expediente      0   
1                  fecha      0                 fecha      0   
2                   hora      0                  hora      0   
3           cod_distrito      0          cod_distrito      0   
4         tipo_accidente      0        tipo_accidente      0   
5   estado_meteorologico      0  estado_meteorologico      0   
6          tipo_vehiculo    155         tipo_vehiculo    120   
7           tipo_persona      0          tipo_persona      0   
8             rango_edad      0            rango_edad      0   
9                   sexo      0                  sexo      0   
10         cod_lesividad  21660         cod_lesividad  14696   
11      coordenada_x_utm      0      coordenada_x_utm      0   
12      coordenada_y_utm      0      coordenada_y_utm      0   
13      positiva_alcohol      0      positiva_alcohol      0   
14        positiva_droga      0        positiva_droga      0   
15             ano_fecha      0             ano_fecha      0   

                Nan_2021         
                   index      0  
0         num_expediente      0  
1                  fecha      0  
2                   hora      0  
3           cod_distrito      0  
4         tipo_accidente      0  
5   estado_meteorologico      0  
6          tipo_vehiculo     46  
7           tipo_persona      0  
8             rango_edad      0  
9                   sexo      0  
10         cod_lesividad  18785  
11      coordenada_x_utm      0  
12      coordenada_y_utm      0  
13      positiva_alcohol      0  
14        positiva_droga      0  
15             ano_fecha      0

Ahora solo tendremos Nans en las categorias 'tipo_vehiculo', 'cod_lesividad' y 'tipo_lesividad', pero antes de eliminarlas, vamos a analizar sus categorias.

In [38]:
veh_type_2019 = acc_2019['tipo_vehiculo'].unique().tolist()
veh_type_2020 = acc_2020['tipo_vehiculo'].unique().tolist()
veh_type_2021 = acc_2021['tipo_vehiculo'].unique().tolist()

In [39]:
veh_type_2019_df = pd.DataFrame(veh_type_2019, columns=['Type_2019'])
veh_type_2020_df = pd.DataFrame(veh_type_2020, columns=['Type_2020'])
veh_type_2021_df = pd.DataFrame(veh_type_2021, columns=['Type_2021'])

In [40]:
type_veh_19_20 = pd.merge(left=veh_type_2019_df,right=veh_type_2020_df, left_on='Type_2019', right_on='Type_2020',
                         how='outer')
type_veh_19_20_21 = pd.merge(left=type_veh_19_20,right=veh_type_2021_df, left_on='Type_2020', right_on='Type_2021',
                            how='outer')
type_veh_19_20_21

,Type_2019,Type_2020,Type_2021
0,Furgoneta,Furgoneta,Furgoneta
1,Turismo,Turismo,Turismo
2,Autobús,Autobús,Autobús
3,Ciclomotor,Ciclomotor,Ciclomotor
4,Motocicleta > 125cc,Motocicleta > 125cc,Motocicleta > 125cc
5,Motocicleta hasta 125cc,Motocicleta hasta 125cc,Motocicleta hasta 125cc
6,Todo terreno,Todo terreno,Todo terreno
7,Bicicleta,Bicicleta,Bicicleta
8,Camión rígido,Camión rígido,Camión rígido
9,Maquinaria de obras,Maquinaria de obras,Maquinaria de obras


Como vemos , hay una categoria indefinida a la que asignaremos todos los Nans.

In [41]:
acc_2019 = acc_2019.fillna({'tipo_vehiculo': 'Sin especificar'})
acc_2020 = acc_2020.fillna({'tipo_vehiculo': 'Sin especificar'})
acc_2021 = acc_2021.fillna({'tipo_vehiculo': 'Sin especificar'})

Vemos que los tres datasets tienen ahora las mismas categorias, pero no todos tienen todas. Con la finalidad de crear un unico dataset, todos tienen que ser comparables y compartir las mismas categorias, por lo que para aquellas categorias que no esten incluidas en todos los datasets procederemos a eliminarlas, al entender que los accidentes acontecidos con dichos tipos de vehiculos no fueron recogidos en aquellos años donde no han sido incluidos.

In [42]:
veh_type_2019 = acc_2019['tipo_vehiculo'].unique().tolist()
veh_type_2020 = acc_2020['tipo_vehiculo'].unique().tolist()
veh_type_2021 = acc_2021['tipo_vehiculo'].unique().tolist()

In [43]:
veh_type = veh_type_2019 + veh_type_2020 + veh_type_2021

In [44]:
def mycounter(series):
    return Counter(list(series))

In [45]:
veh_type_freq = mycounter(veh_type)

In [46]:
type_veh_del = list({x: count for x, count in veh_type_freq.items() if count < 3})

In [47]:
type_veh_ok = list({x: count for x, count in veh_type_freq.items() if count == 3})

In [48]:
vehicle = pd.DataFrame(type_veh_ok, columns = ['Type'])

In [49]:
type_veh_del

['Microbús <= 17 plazas',
 'Tranvía',
 'Caravana',
 'Moto de tres ruedas > 125cc',
 'Tren/metro',
 'Ambulancia SAMUR',
 'Moto de tres ruedas hasta 125cc',
 'Ciclomotor de dos ruedas L1e-B',
 'Maquinaria agrícola',
 'Autobús articulado EMT',
 'Ciclomotor de tres ruedas',
 'Ciclo de motor L1e-A']

In [50]:
veh_type_del_2019 = [x for x in veh_type_2019 if x in type_veh_del]
veh_type_del_2020 = [x for x in veh_type_2020 if x in type_veh_del]
veh_type_del_2021 = [x for x in veh_type_2021 if x in type_veh_del]

In [51]:
acc_veh_2019 = acc_2019.copy()
acc_veh_2020 = acc_2020.copy()
acc_veh_2021 = acc_2021.copy()

In [52]:
acc_veh_2019['is_in_2019'] = acc_veh_2019['tipo_vehiculo'].isin(veh_type_del_2019)
acc_veh_2020['is_in_2020'] = acc_veh_2020['tipo_vehiculo'].isin(veh_type_del_2020)
acc_veh_2021['is_in_2021'] = acc_veh_2021['tipo_vehiculo'].isin(veh_type_del_2021)

In [53]:
index_is_in_19 = acc_veh_2019[acc_veh_2019['is_in_2019']==True].index
acc_veh_2019.drop(index_is_in_19,inplace=True)
acc_veh_2019.drop(['is_in_2019'],axis=1,inplace=True)

In [54]:
index_is_in_20 = acc_veh_2020[acc_veh_2020['is_in_2020']==True].index
acc_veh_2020.drop(index_is_in_20,inplace=True)
acc_veh_2020.drop(['is_in_2020'],axis=1,inplace=True)

In [55]:
index_is_in_21 = acc_veh_2021[acc_veh_2021['is_in_2021']==True].index
acc_veh_2021.drop(index_is_in_21,inplace=True)
acc_veh_2021.drop(['is_in_2021'],axis=1,inplace=True)

Vamos a comprobar que todas las categorias son coincidentes...

In [56]:
veh_type_2019 = acc_veh_2019['tipo_vehiculo'].unique().tolist()
veh_type_2020 = acc_veh_2020['tipo_vehiculo'].unique().tolist()
veh_type_2021 = acc_veh_2021['tipo_vehiculo'].unique().tolist()

In [57]:
veh_type_2019_df = pd.DataFrame(veh_type_2019, columns=['Type_2019'])
veh_type_2020_df = pd.DataFrame(veh_type_2020, columns=['Type_2020'])
veh_type_2021_df = pd.DataFrame(veh_type_2021, columns=['Type_2021'])

In [58]:
type_veh_19_20 = pd.merge(left=veh_type_2019_df,right=veh_type_2020_df, left_on='Type_2019', right_on='Type_2020',
                         how='outer')
type_veh_19_20_21 = pd.merge(left=type_veh_19_20,right=veh_type_2021_df, left_on='Type_2020', right_on='Type_2021',
                            how='outer')
type_veh_19_20_21_type = pd.merge(left=type_veh_19_20_21,right=vehicle, left_on='Type_2021', right_on='Type',
                            how='outer')
type_veh_19_20_21_type

,Type_2019,Type_2020,Type_2021,Type
0,Furgoneta,Furgoneta,Furgoneta,Furgoneta
1,Turismo,Turismo,Turismo,Turismo
2,Autobús,Autobús,Autobús,Autobús
3,Ciclomotor,Ciclomotor,Ciclomotor,Ciclomotor
4,Motocicleta > 125cc,Motocicleta > 125cc,Motocicleta > 125cc,Motocicleta > 125cc
5,Motocicleta hasta 125cc,Motocicleta hasta 125cc,Motocicleta hasta 125cc,Motocicleta hasta 125cc
6,Todo terreno,Todo terreno,Todo terreno,Todo terreno
7,Bicicleta,Bicicleta,Bicicleta,Bicicleta
8,Camión rígido,Camión rígido,Camión rígido,Camión rígido
9,Maquinaria de obras,Maquinaria de obras,Maquinaria de obras,Maquinaria de obras


Ahora vamos a hacer lo mismo con la variable 'cod_lesividad':

In [59]:
cod_lesividad_2019 = acc_2019['cod_lesividad'].unique().tolist()
cod_lesividad_2020 = acc_2020['cod_lesividad'].unique().tolist()
cod_lesividad_2021 = acc_2021['cod_lesividad'].unique().tolist()

In [60]:
cod_lesividad_2019_df = pd.DataFrame(cod_lesividad_2019, columns=['Type_2019'])
cod_lesividad_2020_df = pd.DataFrame(cod_lesividad_2020, columns=['Type_2020'])
cod_lesividad_2021_df = pd.DataFrame(cod_lesividad_2021, columns=['Type_2021'])

In [61]:
cod_lesividad_19_20 = pd.merge(left=cod_lesividad_2019_df,right=cod_lesividad_2020_df, left_on='Type_2019',\
                               right_on='Type_2020',how='outer')
cod_lesividad_19_20_21 = pd.merge(left=cod_lesividad_19_20,right=cod_lesividad_2021_df, left_on='Type_2020',\
                                  right_on='Type_2021', how='outer')
cod_lesividad_19_20_21

,Type_2019,Type_2020,Type_2021
0,NaN,NaN,NaN
1,2.0,2.0,2.0
2,14.0,14.0,14.0
3,5.0,5.0,5.0
4,7.0,7.0,7.0
5,3.0,3.0,3.0
6,1.0,1.0,1.0
7,6.0,6.0,6.0
8,4.0,4.0,4.0
9,77.0,77.0,NaN


Por la descripción del dataset sabemos que los datos en los que no se ha incluido informacion significa que coinciden con la categoria '14' - Sin asistencia sanitaria', por lo que cambiaremos los Nans a dicha categoria.

In [62]:
acc_veh_2019_df = acc_veh_2019.fillna({'cod_lesividad': 14})
acc_veh_2020_df = acc_veh_2020.fillna({'cod_lesividad': 14})
acc_veh_2021_df = acc_veh_2021.fillna({'cod_lesividad': 14})

**3. UNIÓN DE LOS DATASETS**


In [63]:
acc_veh = pd.concat([acc_veh_2019_df,acc_veh_2020_df,acc_veh_2021_df]) 

**4. REDUCCIÓN DEL TAMAÑO** - punto de control

In [64]:
print('ORIGINAL DS:')
print(accidents_2019.shape)
print('RENAME COL.:')
print(accidents_2019_df.shape)
print('ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:')
print(accidents_2019_df_clean.shape)
print('PROCESS. & ELIM. NANS:')
print(acc_2019.shape)
print('ELIM. DIF VEH.:')
print(acc_veh_2019.shape)
print('ELIM. DIF cod_les & tipo_les.:')
print(acc_veh_2019_df.shape)

ORIGINAL DS:
(51811, 19)
RENAME COL.:
(51811, 19)
ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:
(51809, 16)
PROCESS. & ELIM. NANS:
(51700, 16)
ELIM. DIF VEH.:
(51696, 16)
ELIM. DIF cod_les & tipo_les.:
(51696, 16)


In [65]:
print('ORIGINAL DS:')
print(accidents_2020.shape)
print('RENAME COL.:')
print(accidents_2020_df.shape)
print('ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:')
print(accidents_2020_df_clean.shape)
print('PROCESS. & ELIM. NANS:')
print(acc_2020.shape)
print('ELIM. DIF VEH.:')
print(acc_veh_2020.shape)
print('ELIM. DIF cod_les & tipo_les.:')
print(acc_veh_2020_df.shape)

ORIGINAL DS:
(32433, 19)
RENAME COL.:
(32433, 19)
ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:
(32431, 16)
PROCESS. & ELIM. NANS:
(32325, 16)
ELIM. DIF VEH.:
(32296, 16)
ELIM. DIF cod_les & tipo_les.:
(32296, 16)


In [66]:
print('ORIGINAL DS:')
print(accidents_2021.shape)
print('RENAME COL.:')
print(accidents_2021_df.shape)
print('ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:')
print(accidents_2021_df_clean.shape)
print('PROCESS. & ELIM. NANS:')
print(acc_2021.shape)
print('ELIM. DIF VEH.:')
print(acc_veh_2021.shape)
print('ELIM. DIF cod_les & tipo_les.:')
print(acc_veh_2021_df.shape)

ORIGINAL DS:
(41783, 19)
RENAME COL.:
(41783, 19)
ELIM. COL + ELIM. DIF. YEAR + INCLU. COL. YEAR:
(41778, 16)
PROCESS. & ELIM. NANS:
(41596, 16)
ELIM. DIF VEH.:
(41557, 16)
ELIM. DIF cod_les & tipo_les.:
(41557, 16)


**5. NUEVAS VARIABLES** - Vamos a incluir las siguientes columnas:
- dia de la semana (dia_sem)
- mes (mes)
- nº de partes por accidente (num_exp)
- hora (hora_H)
- fin de semana (es_finde)
- festivo (es_fest)
- fecha en formatro datetime (fecha_datetime)

In [67]:
def findDay(date): 
    date_x = datetime.datetime.strptime(date, '%d/%m/%Y').weekday() 
    return (calendar.day_name[date_x]) 

In [68]:
acc_veh['dia_sem']=acc_veh['fecha'].apply(findDay) 
acc_veh['mes']=acc_veh['fecha'].str[3:5].astype(int)
count_exp = acc_veh.groupby('num_expediente')['fecha'].count().reset_index()
count_exp = count_exp.rename(columns={'fecha':'num_exp'})
acc_veh_final = pd.merge(acc_veh,count_exp )

In [69]:
acc_veh_final['hora_H'] =acc_veh_final['hora'].str[:-6].astype(int)

Vamos a aplicar un label encoder para transformar la columna dia de la semana a número

In [70]:
dia_sem_dic = {'Monday':1,
                'Tuesday':2,
                'Wednesday':3,
                'Thursday':4,
                'Friday':5,
                'Saturday':6,
                'Sunday':7}

In [71]:
acc_veh_final['dia_sem_code'] = acc_veh_final.dia_sem.map(dia_sem_dic)

Vamos a incluir el dataset de festivos en la ciudad de Madrid

In [72]:
path_bank_holidays = '/home/dsc/Master DS/TFM/GIT/DATOS_BRUTOS/calendario.csv'

In [73]:
bank_holidays = pd.read_csv(path_bank_holidays, sep = ';' )

In [74]:
bank_holidays['fecha_datetime'] = pd.to_datetime(bank_holidays['Día'], dayfirst=True)

In [75]:
filtered_df =bank_holidays.loc[bank_holidays["fecha_datetime"].between('2019-01-01', '2021-12-31')]

In [76]:
filtered_df['laborable / festivo / domingo festivo'].unique()

array(['festivo', 'laborable', 'sábado', 'domingo', nan, 'Festivo'],
      dtype=object)

In [77]:
bank_holidays_df = filtered_df[(filtered_df['laborable / festivo / domingo festivo'] == 'festivo')|\
                           (filtered_df['laborable / festivo / domingo festivo'] == 'Festivo')]

In [78]:
bank_holidays_list = list(bank_holidays_df['Día'])

In [79]:
acc_veh_final['es_finde'] = (acc_veh_final['dia_sem'] == 'Saturday') |\
                                    (acc_veh_final['dia_sem'] == 'Sunday')

In [80]:
acc_veh_final['es_fest'] = (acc_veh_final['fecha']).isin(bank_holidays_list)

In [81]:
acc_veh_final['fecha_datetime'] = pd.to_datetime(acc_veh_final['fecha'], format='%d/%m/%Y')

**6. UTMs** - Transformamos la columna de UTMs a numeros y los corregimos en unidad de medida:

In [82]:
acc = acc_veh_final.copy()

In [83]:
acc["coordenada_x_utm"] = acc['coordenada_x_utm'].str.replace('.','')
acc["coordenada_y_utm"] = acc['coordenada_y_utm'].str.replace('.','')

/tmp/ipykernel_10582/3111570593.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc["coordenada_x_utm"] = acc['coordenada_x_utm'].str.replace('.','')
/tmp/ipykernel_10582/3111570593.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc["coordenada_y_utm"] = acc['coordenada_y_utm'].str.replace('.','')


In [84]:
acc['coordenada_x_utm'] = pd.to_numeric(acc['coordenada_x_utm'],errors = 'coerce')
acc['coordenada_y_utm'] = pd.to_numeric(acc['coordenada_y_utm'],errors = 'coerce')

In [85]:
def divide_1000(x):
    return x/1000

In [86]:
acc['coordenada_x_utm'] = acc['coordenada_x_utm'].apply(divide_1000)
acc['coordenada_y_utm'] = acc['coordenada_y_utm'].apply(divide_1000)

Al comprobar los valores de 'coordenada_x_utm' y 'coordenada_y_utm' vemos que algunos faltan al asignarles el valor 0. Eliminamos esas lineas.

In [87]:
acc.drop(acc[acc['coordenada_x_utm']==0].index,axis=0, inplace = True)

Incluimos dos columnas con la latitud y la longitud.

In [88]:
!pip install utm

In [89]:
import utm

In [90]:
def getLATLONG(row):
    tup = utm.to_latlon(row[0],row[1],30,'N')
    return pd.Series(tup[:2])

In [91]:
acc[['lat','lon']] = acc[['coordenada_x_utm','coordenada_y_utm']].apply(getLATLONG , axis=1)


**7. Estado Meteorológico Desconocido**

Vamos a intentar eliminar los 'Se desconoce' al considerar que para dichos dias en otros siniestros sí habrá sido notificado el 'estado_meteorologico' y pondremos el estado más frecuente de los comunicados para dicho día

In [92]:
meteo_desc = acc[acc['estado_meteorologico']=='Se desconoce']

In [93]:
meteo_conoc = acc[acc['estado_meteorologico']!='Se desconoce']

In [94]:
meteo_conoc['estado_meteorologico_sin_desc']=meteo_conoc['estado_meteorologico']

/tmp/ipykernel_10582/435545685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meteo_conoc['estado_meteorologico_sin_desc']=meteo_conoc['estado_meteorologico']


In [95]:
est_meteo_freq = meteo_conoc.groupby(['fecha_datetime'])['estado_meteorologico']\
                            .apply(lambda x: x.value_counts().index[0]).reset_index()

In [96]:
meteo_desc_trans = pd.merge(meteo_desc, est_meteo_freq, left_on ='fecha_datetime', right_on ='fecha_datetime')

In [97]:
meteo_desc_trans_df = meteo_desc_trans.drop(['estado_meteorologico_x'], axis=1)

In [98]:
meteo_desc_trans_df = meteo_desc_trans_df.rename(columns={'estado_meteorologico_y':'estado_meteorologico_sin_desc'})

In [99]:
acc = pd.concat([meteo_conoc,meteo_desc_trans_df])

In [100]:
acc['estado_meteorologico'].fillna('Se desconoce', inplace = True)

**8. Pasar codigo de distrito a formato string para la visualización**

In [101]:
acc['cod_dist_string']=acc['cod_distrito'].astype(int).astype(str)

## DATASET DEL NUMERO DE ACCIDENTES

En este punto del proceso, una vez que hemos creado un dataset unico donde toda la informacion ha sido unificada, vamos a tomar dos rutas, cada una con un dataset distinto:

**A) accidentes.csv**

Sabemos por la descripcion del dataset original que para un mismo 'num_expediente' que tendremos una linea por persona involucrada en el accidente, es por ello por lo que hemos creado la variable 'num_exp'. Con esta variable tendremos la información de la escala del siniestro en terminos de numero de personas involucradas en el mismo, lo veremos como el numero de recursos que tendrán que acudir al accidente.

Como nostros solo queremos conocer el numero de siniestros que han ocurrido en un intervalo de tiempo, vamos a crear un dataframe que incluya los valores unicos de 'num_expediente'.  

Este dataset mantendrá las columnas para aquellas lineas (con el mismo 'num_expediente') que sean iguales, al describir el accidente:
- num_expediente
- fecha
- fecha_datetime
- hora
- cod_distrito
- tipo_accidente
- estado_meteorologico
- estado_meteorologico_sin_desc
- coordenada_x_utm
- coordenada_y_utm
- lat
- lon
- ano_fecha
- dia_sem
- mes
- hora
- num_exp
- es_finde
- es_fest

**B) accidentes_total.csv**

Como nosotros también queremos un dataset en el que se describan las personas involucradas en el accidente, vamos a mantener el dataset completo.

Adicionalmente al dataset anterior, incluiremos las siguientes variables:
- tipo_vehiculo
- tipo_persona
- rango_edad
- cod_lesividad
- tipo_lesividad
- positiva_alcohol
- positiva_droga
- sexo

In [102]:
fea_to_drop = ['tipo_vehiculo','tipo_persona', 'rango_edad', 'sexo','positiva_alcohol','positiva_droga',\
               'cod_lesividad']

In [103]:
acc_pre = acc.drop(fea_to_drop, axis=1)

In [104]:
acc_unique = acc_pre.drop_duplicates(subset = None) 

In [105]:
lista = acc_unique['num_expediente'].unique()

Vamos a comprobar que no hay duplicados en 'num_expediente'...

In [106]:
acc_unique.shape[0] - len(lista)

0

## GUARDAMOS LOS DATOS PROCESADOS EN ARCHIVOS

Guardaremos los datasets en archivos csv para futuros análisis.

**1. 'accidentes.csv':** Dataset de siniestros únicos

In [107]:
acc_unique.to_csv(r'/home/dsc/Master DS/TFM/GIT/DATOS_PROCESADOS/accidentes.csv',sep='^',\
                  index = False, header = True)

**2. 'accidentes_total.csv':** Dataset completo

In [108]:
acc.to_csv(r'/home/dsc/Master DS/TFM/GIT/DATOS_PROCESADOS/accidentes_total.csv',sep='^',\
                  index = False, header = True)

**3. 'cod_distrito.csv':** Tabla normalizada de información de distritos

In [109]:
dist_cod.to_csv(r'/home/dsc/Master DS/TFM/GIT/DATOS_PROCESADOS/cod_distrito.csv',sep='^',\
                  index = False, header = True)

**4. 'cod_lesiv.csv':** Tabla normalizada de la lesividad de los siniestros

In [110]:
lesiv_cod_type.to_csv(r'/home/dsc/Master DS/TFM/GIT/DATOS_PROCESADOS/cod_lesiv.csv',sep='^',\
                  index = False, header = True)